In [1]:
import torch
import deepspeed

[2024-07-05 01:49:56,141] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/cluster/home/terjenf/.conda/envs/vgdebatt/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
from accelerate import Accelerator
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, Dataset
from datasets import load_dataset, load_from_disk
from tqdm.auto import tqdm
import deepspeed

In [3]:
model_id = "NorLLM-AI/NorMistral-7B"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id,device_map='auto',torch_dtype=torch.bfloat16)                                         
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
tokenizer.pad_token = tokenizer.eos_token

In [6]:
deepspeed.init_distributed(dist_backend="nccl", world_size=2)

[2024-07-05 01:43:00,119] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-07-05 01:43:00,119] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-07-05 01:43:00,195] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.1.4.1, master_port=29500
[2024-07-05 01:43:00,195] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [7]:
# Initialize the DeepSpeed-Inference engine
ds_engine = deepspeed.init_inference(model,
                                 tensor_parallel={"tp_size": 1},
                                 dtype=torch.half,

                                 replace_with_kernel_inject=True)


[2024-07-05 01:50:34,460] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.2, git-hash=unknown, git-branch=unknown
[2024-07-05 01:50:34,462] [INFO] [logging.py:96:log_dist] [Rank 0] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


You shouldn't move a model when it is dispatched on multiple devices.


In [18]:
tokens = tokenizer("hei, jeg heter terje", max_length=20, return_tensors='pt')
tokens

{'input_ids': tensor([[    1, 36512, 28725, 32008, 23402,  3636,  2099]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [19]:
type(model)

transformers.models.mistral.modeling_mistral.MistralForCausalLM

In [27]:
model.hf_device_map

OrderedDict([('model.embed_tokens', 0),
             ('model.layers.0', 0),
             ('model.layers.1', 0),
             ('model.layers.2', 0),
             ('model.layers.3', 0),
             ('model.layers.4', 0),
             ('model.layers.5', 0),
             ('model.layers.6', 0),
             ('model.layers.7', 0),
             ('model.layers.8', 0),
             ('model.layers.9', 0),
             ('model.layers.10', 0),
             ('model.layers.11', 0),
             ('model.layers.12', 0),
             ('model.layers.13', 0),
             ('model.layers.14', 0),
             ('model.layers.15', 0),
             ('model.layers.16', 1),
             ('model.layers.17', 1),
             ('model.layers.18', 1),
             ('model.layers.19', 1),
             ('model.layers.20', 1),
             ('model.layers.21', 1),
             ('model.layers.22', 1),
             ('model.layers.23', 1),
             ('model.layers.24', 1),
             ('model.layers.25', 1),
        

In [40]:
model.generate(input_ids=tokens['input_ids'].to("cuda:0"), attention_mask=tokens['attention_mask'].to("cuda:0"))
print("hei")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [38]:
model(input_ids=tokens['input_ids'].to("cuda:0"), attention_mask=tokens['attention_mask'].to("cuda:0"), labels=tokens['input_ids'].to("cuda:1"))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [33]:
model(input_ids=tokens['input_ids'].to(torch.device('cuda')), attention_mask=tokens['attention_mask'].to(torch.device('cuda')))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [14]:
type(ds_engine)

deepspeed.inference.engine.InferenceEngine

In [8]:
deepspeed_path = "/cluster/home/terjenf/norwAI_All/results/Checkpoints_NRK_Peft_NorMistral/checkpoint-28000"

In [48]:
ds_engine.load_checkpoint(deepspeed_path)

device(type='cuda', index=0)

In [49]:
ds_engine.module.device

device(type='cuda', index=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': [1, 36512, 28725, 32008, 23402, 3636, 2099], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [10]:
model(input_ids=tokens['input_ids'], tokens['attention_mask'])

SyntaxError: positional argument follows keyword argument (1990286007.py, line 1)

In [40]:
detokenized = tokenizer.decode(tokens['input_ids'])
detokenized

'<s> hei, jeg heter terje'

In [41]:
output = model(tokens)

AttributeError: 